In [1]:
from dask.distributed import Client, progress, LocalCluster

# client = Client("tcp://127.0.0.1:50328")
# client = Client(processes=False, threads_per_worker=3, n_workers=3, memory_limit="10GB")
# client = Client(n_workers=4)
# client = Client()
# cluster = LocalCluster()
# client = Client(cluster)
client = Client("tcp://192.168.254.34:8786")

client

Client Scheduler: tcp://192.168.254.34:8786 Dashboard: http://192.168.254.34:8787/status,Cluster Workers: 2 Cores: 24 Memory: 53.34 GB


In [2]:
client.get_versions(check=True)

{'scheduler': {'host': {'python': '3.8.2.final.0',
   'python-bits': 64,
   'OS': 'Linux',
   'OS-release': '4.19.104-microsoft-standard',
   'machine': 'x86_64',
   'processor': 'x86_64',
   'byteorder': 'little',
   'LC_ALL': 'None',
   'LANG': 'C.UTF-8'},
  'packages': {'python': '3.8.2.final.0',
   'dask': '2.27.0',
   'distributed': '2.27.0',
   'msgpack': '1.0.0',
   'cloudpickle': '1.6.0',
   'tornado': '6.0.4',
   'toolz': '0.10.0',
   'numpy': '1.19.2',
   'lz4': None,
   'blosc': None}},
 'workers': {'tcp://192.168.254.34:32963': {'host': {'python': '3.8.2.final.0',
    'python-bits': 64,
    'OS': 'Linux',
    'OS-release': '4.19.104-microsoft-standard',
    'machine': 'x86_64',
    'processor': 'x86_64',
    'byteorder': 'little',
    'LC_ALL': 'None',
    'LANG': 'C.UTF-8'},
   'packages': {'python': '3.8.2.final.0',
    'dask': '2.27.0',
    'distributed': '2.27.0',
    'msgpack': '1.0.0',
    'cloudpickle': '1.6.0',
    'tornado': '6.0.4',
    'toolz': '0.10.0',
    'num

client.restart()

In [3]:
import dask
import dask.dataframe as dd

In [4]:
df = dd.read_parquet('local_table3.parquet.gzip', engine='pyarrow')
df

,personid,status,finger_index,cylindersbinary
npartitions=1,,,,
,int64,object,object,object
,...,...,...,...


In [5]:
print(df.compute())
print(df.divisions)
print(df.npartitions)
print(df.ndim)

    personid  status  finger_index  \
0     365538       1            11   
1     365538       1            12   
2     365538       1            13   
3     365538       1            14   
4     365538       1            15   
..       ...     ...           ...   
8     413208       1            19   
9     413208       1            20   
0     186584       1            11   
1     186584       1            12   
2     186584       1            13   

                                      cylindersbinary  
0   b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...  
1   b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...  
2   b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...  
3   b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...  
4   b'\x00\x00\xff\x00\x00\x00\xff\x0f\x00\xf0\xff...  
..                                                ...  
8   b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...  
9   b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...  
0   b'\x00\xff\xff\x00\xf0\xff\xff\x00\

# df = df.repartition(npartitions=df.npartitions // 100)
df = df.repartition(npartitions=4)

In [6]:
df.index

Dask Index Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: read-parquet, 2 tasks

In [7]:
len(df)

13150

In [8]:
type(df)

dask.dataframe.core.DataFrame

In [9]:
sourceData = df[(df.personid == 365538) & (df.finger_index == 11)]
sourceData = sourceData['cylindersbinary'].compute().loc[0]
# sourceData

sourceData = df.loc(365538)
sourceData = sourceData[df.finger_index == 11]
sourceData = sourceData['cylindersbinary'].compute().loc[0]
# sourceData

df = df.set_index('personid')

In [10]:
df = client.persist(df)
df

,personid,status,finger_index,cylindersbinary
npartitions=1,,,,
,int64,object,object,object
,...,...,...,...


In [11]:
type(df)

dask.dataframe.core.DataFrame

In [12]:
df

,personid,status,finger_index,cylindersbinary
npartitions=1,,,,
,int64,object,object,object
,...,...,...,...


In [13]:
df = df.tail(100)
df = dd.from_pandas(df, npartitions=2)

In [14]:
type(sourceData)

bytes

In [15]:
len(df)

100

In [16]:
df.compute().head(36)

,personid,status,finger_index,cylindersbinary
0,436479,1,11,b'\x00\xff\x00\x00\xf0\xff\xff\x00\xff\xff\xff...
0,186584,1,11,b'\x00\xff\xff\x00\xf0\xff\xff\x00\xff\xff\xff...
0,189217,1,16,b'\x00\xff\xff\x00\xf0\xff\xff\x00\xff\xff\xff...
0,372153,1,11,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
0,376918,1,11,b'\x00\xff\xff\x00\xf0\xff\xff\xff\xff\xff\xff...
0,413208,1,11,b'\xff\xff\xff\x00\x00\xff\xff\x0f\x00\xff\xff...
0,139435,1,11,b'\x00\x00\xff\x00\x00\x00\xff\x0f\x00\xf0\xff...
0,347742,1,11,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
0,181955,1,11,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...
0,391456,1,11,b'\x00\xff\xff\x00\xf0\xff\xff\x0f\xff\xff\xff...


In [17]:
import ctypes as ct
from ctypes import cdll

def compareFing(a,b,c,d):
    baseDir = '/home/dev/Practice/MCCLib-DASK'
    # ct.CDLL._func_restype_ = ct.c_float

    libComp = cdll.LoadLibrary(baseDir + '/build/bin/fingercomp.so')

    compareF = libComp.compareFingers
    compareF.argtypes = [ct.c_char_p, ct.c_long, ct.c_char_p, ct.c_long]
    compareF.restype = ct.c_float
    ret = compareF(a,b,c,d)
    
    return ret 

In [18]:
def calcWrapper(row):
#     print("type(row) => ", type(row))
#     print("row.dtypes => ", row.dtypes)
#     print("row['Last Name'] => ", row['Last Name'])
#     print("row => \n", row)

#     return libComp.compareFingers(ct.c_char_p(sourceData),
#                                            len(sourceData),
#                                            ct.c_char_p(bytes(row['cylindersbinary'])),
#                                            len(row['cylindersbinary']))
#     print(type(row['cylindersbinary']))

#     return libComp.compareFingers(sourceData,
#                                            len(sourceData),
#                                            sourceData,
#                                            len(sourceData))

#     return libComp.compareFingers(ct.c_char_p(sourceData),
#                                            len(sourceData),
#                                            ct.c_char_p(row['cylindersbinary']),
#                                            len(row['cylindersbinary']))

    return compareFing(sourceData,
                       len(sourceData),
                       row['cylindersbinary'],
                       len(row['cylindersbinary']))

# # a = peopleDaskDataFrame.applymap(myF)
# a = peopleDaskDataFrame.apply(calcWrapper, axis=1)

In [19]:
resultsSeries = df.apply(calcWrapper, axis=1, meta=(None, 'f8'))

In [20]:
type(df)

dask.dataframe.core.DataFrame

In [21]:
type(resultsSeries)

dask.dataframe.core.Series

In [22]:
resultsSeries.compute()

KilledWorker: ("('apply-215ea5811d8781d1fe89eb39b79e2e25', 0)", <Worker 'tcp://192.168.254.34:32797', name: tcp://192.168.254.34:32797, memory: 0, processing: 3>)

In [1]:
client.persist(resultsSeries)

NameError: name 'client' is not defined

In [31]:
%%time
resultsSeries.head()

CPU times: user 5.92 ms, sys: 2.63 ms, total: 8.55 ms
Wall time: 20.5 ms


0    0.8684
0    0.1321
0    0.1324
1    0.1310
1    0.1199
dtype: float64

In [34]:
len(resultsSeries)

30

In [33]:
resultsSeries.nlargest(15).compute()

0    0.8684
3    0.1493
2    0.1443
7    0.1368
4    0.1334
0    0.1324
0    0.1321
1    0.1310
1    0.1267
5    0.1249
2    0.1244
3    0.1244
4    0.1244
2    0.1241
6    0.1238
dtype: float64

In [22]:
resultsSeries.compute()

AttributeError: type object 'PyCFuncPtrType' has no attribute '__mul__'